## Part 2: No edits from here
#### You can't edit this part, and neither me.
#### Note: Maybe I can edit if there is a major bug

These are the functions for running and reporting

In [1]:
import numpy as np
from vec_db import VecDB
import time
from dataclasses import dataclass
from typing import List
from memory_profiler import memory_usage
import gc

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

results = []
to_print_arr = []

def run_queries(db, query, top_k, actual_ids, num_runs):
    global results
    results = []
    for _ in range(num_runs):
        tic = time.time()
        db_ids = db.retrive(query, top_k)
        toc = time.time()
        run_time = toc - tic
        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results

def memory_usage_run_queries(args):
    global results
    # This part is added to calcauate the RAM usage
    mem_before = max(memory_usage())
    mem = memory_usage(proc=(run_queries, args, {}), interval =  1e-6)
    return results, max(mem) - mem_before

def evaluate_result(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

def get_actual_ids_first_k(actual_sorted_ids, k):
    return [id for id in actual_sorted_ids if id < k]

This to generate 10K database and the query using the seed numbers that will be changed at submissions day

In [2]:
DB_SEED_NUMBER=50
rng = np.random.default_rng(DB_SEED_NUMBER)
vectors = rng.random((2*10**4+5*10**3, 70), dtype=np.float32)
# vectors = rng.random((2*10**3, 70), dtype=np.float32)
QUERY_SEED_NUMBER=50
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)

actual_sorted_ids_10k = np.argsort(vectors.dot(query.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]

Open new DB add 10K then retrieve and evaluate. Then add another 90K (total 100K) then retrieve and evaluate.

In [51]:
db = VecDB()
# db.L,db.R= VecDB.hundredKparams
# db.RecordsPerCluster=1*10**3
# for 20k (16,17) is ok but -125 max recall and around -36 and 16 etc
# 17 17 is same same 
# 18 17 seems to be very good, max is -40
# 17 18 is worse
# (19,17) is bad
#(18,19) is bad
#(18,20) is bad
# trialParams=(18,17) #default is 15,17
trialParams=(19,17) #default is 15,17
db.L,db.R= trialParams
# db.L,db.R=VecDB.fiveKparams
db.n_clusters=1
# print("Records per cluster :",db.RecordsPerCluster)
print("L,R :",db.L,db.R)
print("no. of clusters: ",db.n_clusters)
print("estimated around ",len(vectors)/db.n_clusters," records per cluster")
print("Inserting", len(vectors), "records")
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(vectors)]
db.insert_records(records_dict)
res = run_queries(db, query, 5, actual_sorted_ids_10k, 1) # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_sorted_ids_10k, 5)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"10K\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)

L,R : 19 17
no. of clusters:  1
estimated around  25000.0  records per cluster
Inserting 25000 records


C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


writing to  DBIndex10K/0.bin
10K	score	0.0	time	0.31	RAM	6.21 MB


In [64]:
from vec_db import VecDB
newDB=VecDB("DBIndex10K/",False)
newDB.L, newDB.R= VecDB.tenKparams
query = rng.random((1, 70), dtype=np.float32)
actual_sorted_ids_10k = np.argsort(vectors.dot(query.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]
# res = run_queries(newDB, query, 5, actual_sorted_ids_10k, 1) # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((newDB, query, 5, actual_sorted_ids_10k, 5)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"10K\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)

10K	score	-61.0	time	0.31	RAM	5.25 MB


In [ ]:
import sys
import pickle
mm= pickle.load(open("DBIndex/0.bin","rb"))
print(len(mm.verticies))

20000


In [ ]:
print(res)
print(actual_sorted_ids_10k[:10])

[Result(run_time=0.2322697639465332, top_k=5, db_ids=[0, 0, 661, 661, 7946], actual_ids=[0, 661, 7946, 3422, 7278, 228, 4451, 2779, 8628, 5932, 7932, 2887, 8726, 185, 1115, 4516, 6466, 9167, 4569, 1361, 8795, 5380, 509, 6191, 9589, 689, 7231, 3035, 8912, 3775, 4236, 915, 5921, 3597, 2584, 4584, 9734, 7798, 7093, 9613, 2091, 9319, 2812, 5421, 3884, 3305, 4051, 7137, 4169, 5546, 7834, 7263, 4180, 4696, 8242, 9595, 9799, 8336, 8656, 5231, 852, 3954, 257, 7752, 9793, 4529, 4921, 9081, 7025, 7651, 4385, 9542, 4802, 6809, 599, 2740, 977, 8919, 7758, 2303, 6239, 2118, 7713, 826, 4469, 9640, 4971, 5864, 758, 2100, 4319, 7218, 1930, 3709, 4852, 465, 7642, 146, 2169, 3903, 5807, 715, 4963, 6580, 7241, 5056, 7825, 5635, 7579, 9008, 1623, 8201, 3811, 1876, 3904, 4204, 2843, 2711, 8667, 857, 1386, 2088, 6471, 2319, 8890, 1154, 9548, 4523, 2986, 8963, 8719, 9981, 9176, 4243, 8325, 8202, 1934, 5142, 8020, 1514, 2263, 117, 9970, 7136, 7064, 2873, 2979, 7322, 804, 114, 4448, 8571, 4305, 3992, 9177, 330

Remove exsiting varaibles to empty some RAM

In [ ]:
del vectors
del query
del actual_sorted_ids_10k
del records_dict
del db
gc.collect()

0

This code to generate 20M database. The seed (50) will not be changed. Create the same DB and prepare it's files indexes and every related file. <br>
Note at the submission I'll not run the insert records. <br>
The query istelf will be changed at submissions day but not the DB

In [ ]:
rng = np.random.default_rng(50)
vectors = rng.random((10**7*2, 70), dtype=np.float32)

rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)

actual_sorted_ids_20m = np.argsort(vectors.dot(query.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]

In [ ]:
print("Team Number", TEAM_NUMBER)
print("\n".join(to_print_arr))

db = VecDB(file_path = PATH_DB_100K, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**5)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"100K\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


db = VecDB(file_path = PATH_DB_1M, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"1M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


db = VecDB(file_path = PATH_DB_5M, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*5)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"5M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


db = VecDB(file_path = PATH_DB_10M, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*10)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"10M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


db = VecDB(file_path = PATH_DB_15M, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*15)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"15M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


db = VecDB(file_path = PATH_DB_20M, new_db = False)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*20)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"20M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)


Team Number 1
10K	score	0.0	time	1.50	RAM	0.61 MB
100K	score	-307208.0	time	7.61	RAM	2.61 MB
1M	score	-3062013.0	time	8.51	RAM	0.05 MB
5M	score	-15309954.0	time	7.99	RAM	0.00 MB
10M	score	-30614896.0	time	7.66	RAM	0.01 MB
15M	score	-45922276.0	time	7.43	RAM	0.00 MB
20M	score	-61232472.0	time	7.20	RAM	0.00 MB


In [ ]:
print("Team Number", TEAM_NUMBER)
print("\n".join(to_print_arr))

Team Number 1
10K	score	0.0	time	1.50	RAM	0.61 MB
100K	score	-307208.0	time	7.61	RAM	2.61 MB
1M	score	-3062013.0	time	8.51	RAM	0.05 MB
5M	score	-15309954.0	time	7.99	RAM	0.00 MB
10M	score	-30614896.0	time	7.66	RAM	0.01 MB
15M	score	-45922276.0	time	7.43	RAM	0.00 MB
20M	score	-61232472.0	time	7.20	RAM	0.00 MB
